In [1]:
# Import math functions (numpy) and sympy for creating formulars
from IPython.display import clear_output
import numpy as np
import sympy as sp
from fractions import Fraction
import matplotlib.pyplot as plt

In [2]:
# Import qiskit framework
from qiskit import IBMQ
from qiskit_optimization import QuadraticProgram
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import QAOA

In [3]:
#IBMQ.load_account()
#provider = IBMQ.get_provider(hub='ibm-q')

In [4]:
clear_output()

## Define Functions

In [5]:
def construct_function(kl, kh, n, h):
    """Constructs the mathematical function to be optimized

    Args:
        kl (double): low k
        kh (double): high k
        n (int): number of heads
        h (list): values for each of the heads

    Returns:
        sympyfunction, dict: mathematical sympy function with coefficients
    """
    function = 0
    sp.init_printing(use_unicode=True)
    
    # construct variables for each permeability
    for i in range(1,n):
        globals()['q%s' % i] = sp.symbols('q'+str(i))
        globals()['h%s' % i] = sp.symbols('q'+str(i))
        
    # for each permeability, add its contribution to the total cost function
    for i in range(2,n):
        term = ((kl+globals()['q%s' % (i-1)]*(kh-kl))*(h[i-2]-h[i-1])+(kl+globals()['q%s' % (i)]*(kh-kl))*(h[i]-h[i-1]))**2
        function = function + term
        
    # expand and simplify the cost function
    function = sp.expand(function)
    
    # convert all square terms into linear terms as qi^2 = qi = 0 or 1 for all i and simplify the cost function
    for i in range(1,n):
        function = function.subs(sp.symbols('q'+str(i))**2, (sp.symbols('q'+str(i))))
        coeffs = sp.Poly(function).as_dict()
        
    # function in sympy form and coefficients of all terms in an easily readble dictionary form returned
    return function, coeffs

In [6]:
def generate_QP(coeffs, n, verbose=False):
    """Generates the Quadratic Program to be optimized

    Args:
        coeffs (dict): Contains the coefficients from the sympy funciton
        n (int): Number of heads

    Returns:
        QuadraticProgram: The quadratic program to optimize
    """
    qp = QuadraticProgram()
    for i in range(1,n):
        qp.binary_var('q'+str(i))
    constant = 0
    linear = {}
    quadratic = {}
    for key,value in coeffs.items():
        if sum(key) == 0:
            constant = float(value)
        elif sum(key) == 1:
            term = 'q'+str(np.argmax(key)+1)
            linear[term] = float(value)
        else:
            indices = [i[0] for i in np.argwhere(np.array(key)>0)]
            term = ('q'+str(indices[0]+1),'q'+str(indices[1]+1))
            quadratic[term] = float(value)
    qp.minimize(linear=linear, quadratic=quadratic, constant=constant)
    if verbose:
        print(qp.export_as_lp_string())
    return qp

In [7]:
def hydrologic_inverse_analysis(kl,kh,n,h, verbose=False):
    """Constructs and solves the mathematical function

    Args:
        kl (double): low k
        kh (double): high k
        n (int): number of heads
        h (list): values for each of the heads
        verbose (Bool): Output information or not

    Returns:
        list: list with the estimated k values for the heads
    """
    # Create the mathematical function
    function, coeffs = construct_function(kl,kh,n,h)
    if verbose:
        print('Function in Sympy:', function)
    # Generate the quadratic problem and solve it with Qiskit
    qp = generate_QP(coeffs,n,verbose)
    qins = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1000, seed_simulator=123)
    meo = MinimumEigenOptimizer(min_eigen_solver=QAOA(reps=1, quantum_instance=qins))
    result = meo.solve(qp)
    if verbose:
        print('\nrun time:', result.min_eigen_solver_result.optimizer_time)
        print(result)
    k = [int(i) for i in list(kl+result.x*(kh-kl))]
    if verbose:
        print('k_res: ', k)
    return k

In [8]:
def evaluate_performance(k_true,k_res, verbose=False):
    """Evaluate the performance of the estimation by calculating missmatches and comparing them to the total number of values

    Args:
        k_true (list): True k values
        k_res (list): Estimated k values
        verbose (Bool): Output information or not

    Returns:
        double: Error rate
    """
    total_number = len(k_true)
    false_count = 0
    for i in range(len(k_true)):
        if k_true[i] != k_res[i]:
            false_count += 1
    error_rate = false_count / total_number
    if verbose:
        print('Error Rate = '+ str(error_rate*100)+'%')
    return error_rate

## Small 1D Problem

Given: 3 heads with $h_1=1, h_2=\frac{1}{3}, h_3=0$ <br>
Assumption: $k_l = 1, k_h = 2$ <br>
Task: minimize  $f(q_1, q_2) = \frac{8}{9}q_1 − \frac{1}{9}q_2 − \frac{4}{9}q_1q_2 + \frac{1}{9}$

In [9]:
k_true = [1,2]

In [10]:
kl = 1
kh = 2
n = 3
h = [1,1/3,0]

In [11]:
k_res = hydrologic_inverse_analysis(kl,kh,n,h,verbose=True)

Function in Sympy: -0.444444444444444*q1*q2 + 0.888888888888889*q1 - 0.111111111111111*q2 + 0.111111111111111
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 0.888888888889 q1 - 0.111111111111 q2 + [ - 0.888888888889 q1*q2 ]/2 +
      0.111111111111
Subject To

Bounds
 0 <= q1 <= 1
 0 <= q2 <= 1

Binaries
 q1 q2
End


run time: 0.025502920150756836
optimal function value: 2.7755575615628914e-17
optimal value: [0. 1.]
status: SUCCESS
k_res:  [1, 2]


In [12]:
error_rate = evaluate_performance(k_true, k_res, verbose=True)

Error Rate = 0.0%


## Larger 1D Problem

In [13]:
def generate_random_1D_data(kl, kh, n):
    """Generate random data for the value of the heads

    Args:
        kl (double): low k
        kh (double): high k
        n (int): number of heads

    Returns:
        list, list: Returns the random true k values and the linked head values
    """
    k_true = np.random.randint(2, size=n-1)*(kh-kl) +kl
    
    h1=np.random.rand()
    h2=np.random.rand()
    h=np.array([h1,h2])
    for i in range(1,n-1):
        h_next = h[i]+k_true[i-1]/k_true[i]*(h[i]-h[i-1])
        h=np.append(h, np.array([h_next]))
    return k_true, h

In [14]:
kl = 1
kh = 2
n = 15

In [15]:
# Generate a dataset with true ks and the head values
k_true, h = generate_random_1D_data(kl, kh, n)
print("k =", k_true)
print("h =", h)

k = [2 2 2 2 1 1 2 1 2 2 1 2 2 1]
h = [  0.88968652   0.06050781  -0.76867091  -1.59784963  -2.42702835
  -4.08538578  -5.74374322  -6.57292193  -8.23127937  -9.06045809
  -9.8896368  -11.54799424 -12.37717296 -13.20635167 -14.86470911]


In [16]:
# Estimate the k values, given only the head values
k_res = hydrologic_inverse_analysis(kl,kh,n,h,verbose=True)

Function in Sympy: -1.37507469144248*q1*q2 + 0.687537345721241*q1 - 2.75014938288497*q10*q11 - 1.37507469144248*q10*q9 - 2.75014938288497*q11*q12 + 11.0005975315399*q11 - 1.37507469144248*q12*q13 - 2.75014938288497*q13*q14 + 5.50029876576994*q14 - 1.37507469144248*q2*q3 + 1.37507469144248*q2 - 1.37507469144248*q3*q4 + 1.37507469144248*q3 - 2.75014938288497*q4*q5 + 1.11022302462516e-15*q4 - 5.50029876576993*q5*q6 + 8.2504481486549*q5 - 2.75014938288496*q6*q7 + 8.25044814865489*q6 - 2.75014938288497*q7*q8 - 1.37507469144248*q7 - 2.75014938288497*q8*q9 + 11.0005975315399*q8 + 1.55431223447522e-15*q9 + 4.81276142004869
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 0.687537345721 q1 + 1.375074691442 q2 + 1.375074691442 q3
      + 0.000000000000 q4 + 8.250448148655 q5 + 8.250448148655 q6
      - 1.375074691442 q7 + 11.000597531540 q8 + 0.000000000000 q9
      + 11.000597531540 q11 + 5.500298765770 q14 + [ - 2.750149382885 q1*q2
      - 2

In [17]:
# Compare the estimation
error_rate = evaluate_performance(k_true, k_res, verbose=True)

Error Rate = 28.57142857142857%


## 1D Problem Performance Analysis
Show how the number of heads influences the estimation error

In [ ]:
# Set everything up
runtime = 10
num_heads = 15
error_set = []
avg_set = []
# Start the runtime
for i in range(runtime):
    print("Current run: ", i+1)
    heads = []
    error_rates = []
    kl = 1
    kh = 2
    # Calculate the values for different amounts of heads
    for n in range(3,num_heads+1):
        k_true, h = generate_random_1D_data(kl, kh, n)
        k_res = hydrologic_inverse_analysis(kl, kh, n, h)
        error_rate = evaluate_performance(k_true, k_res)
        heads.append(len(h))
        error_rates.append(error_rate)
    error_set.append(error_rates)
# Do the statisics, take the average for each amount of heads
for i in range(num_heads-2):
    avg = 0
    for j in range(runtime):
        avg+=error_set[j][i]
    avg_set.append(avg/runtime*100)

plt.plot(heads, avg_set)
plt.xlabel("Number of heads")
plt.ylabel("Avg error percentage")
plt.show()

Current run:  1
Current run:  2
Current run:  3
Current run:  4
Current run:  5
